In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

data = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_GYi4Gz5.csv')

# #Replacing Null values of Company_type to Other
data['company_type'].fillna('NpO', inplace = True)
test['company_type'].fillna('NpO', inplace = True)

#replacing with medium company size
data['company_size'].fillna('50-99', inplace  = True)
test['company_size'].fillna('50-99', inplace  = True)

# # #Managing all values accordingly
data.experience[data.experience == '>20'] = 20 
data.experience[data.experience == '<1'] = 1 

test.experience[test.experience == '>20'] = 20 
test.experience[test.experience == '<1'] = 1 

# #Replacing NULL values in experience with median
data['experience'].fillna(9, inplace = True)
test['experience'].fillna(9, inplace = True)

# #Replacing all values in major decipline with 'Other
data['major_discipline'].fillna('Other', inplace = True)
test['major_discipline'].fillna('Other', inplace = True)

# #Handling all the values 
data.last_new_job[data.last_new_job == '>4'] = 5 
test.last_new_job[test.last_new_job == '>4'] = 5 

# #Replscing all the NULL values with Mode
data['last_new_job'].fillna(2, inplace = True)
test['last_new_job'].fillna(2, inplace = True)

# #Replacing all the NULL values in education_evel with mode
data['education_level'].fillna('Masters', inplace = True)
test['education_level'].fillna('Masters', inplace = True) 

# #Replacing all the NULL values in enrolled university with mode
data['enrolled_university'].fillna('Part time course', inplace = True)
test['enrolled_university'].fillna('Part time course', inplace = True)

# #Filling NULL values with mode
data.gender.fillna('Other', inplace =  True) #data['gender'].mode()[0]
test.gender.fillna('Other', inplace =  True) #test['gender'].mode()[0]

X = data.drop(['target'] ,axis = 1)
y = data['target'].copy()
X_test = test

            


X = pd.concat([X.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X['education_level'], prefix='education_level'),
                           pd.get_dummies(X['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X['company_size'], prefix='company_size'),
                           pd.get_dummies(X['company_type'], prefix='company_type')],axis=1)

X_test = pd.concat([X_test.get(['enrollee_id','city','city_development_index', 'gender', 'relevent_experience', 'experience',
                                                                            'last_new_job', 'training_hours' ]),
                           pd.get_dummies(X_test['enrolled_university'], prefix='enrolled_university'),
                           pd.get_dummies(X_test['education_level'], prefix='education_level'),
                           pd.get_dummies(X_test['major_discipline'], prefix='major_discipline'),
                           pd.get_dummies(X_test['company_size'], prefix='company_size'),
                           pd.get_dummies(X_test['company_type'], prefix='company_type')],axis=1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42)

from sklearn import preprocessing


for i in X.columns:
    if X[i].dtype == 'object':
        if len(list(X[i].unique())) >=2:
            #cols.append(i)
            le = preprocessing.LabelEncoder()
            le.fit(X[i].astype(str))
            X[i] = le.transform(X[i].astype(str))
            X_test[i] = le.transform(X_test[i].astype(str))
            #X_val[i] =   le.transform(X_val[i].astype(str))
            #X_test[i] =  le.transform(X_test[i].astype(str))


c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\admin\appdata\local\programs\python\python36\lib\site-p

# Stacking of XGB Models

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

In [9]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [10]:
lgb_params = {}
lgb_params['learning_rate'] = 0.05
lgb_params['n_estimators'] = 650
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 500
lgb_params['seed'] = 99


lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.05
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['seed'] = 99


lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.05
lgb_params3['seed'] = 99



In [11]:
lgb_model = LGBMClassifier(**lgb_params)

lgb_model2 = LGBMClassifier(**lgb_params2)

lgb_model3 = LGBMClassifier(**lgb_params3)

In [12]:
log_model = LogisticRegression()

In [13]:
stack = Ensemble(n_splits=3,
        stacker = log_model,
        base_models = (lgb_model, lgb_model2, lgb_model3))        
        

In [14]:
y_pred = stack.fit_predict(X, y,  X_test)        


Fit LGBMClassifier fold 1
Fit LGBMClassifier fold 2
Fit LGBMClassifier fold 3
Fit LGBMClassifier fold 1
Fit LGBMClassifier fold 2
Fit LGBMClassifier fold 3
Fit LGBMClassifier fold 1
Fit LGBMClassifier fold 2
Fit LGBMClassifier fold 3
Stacker score: 0.64417


In [15]:
y_pred

array([0.49602587, 0.09245333, 0.27886923, ..., 0.12881969, 0.19501813,
       0.1807711 ])

In [16]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')

In [17]:
sub['target'] = y_pred

In [18]:
sub.to_csv('ensembled.csv', index = False)